In [2]:
!pip install transformers datasets evaluate sacrebleu tf-keras ipywidgets sentencepiece sacremoses "accelerate>=0.26.0"

Defaulting to user installation because normal site-packages is not writeable


In [14]:
!pip install nltk bert_score

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 2.5 MB/s eta 0:00:00


In [3]:
# import libraries
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq, Seq2SeqTrainingArguments,
                          Seq2SeqTrainer, EarlyStoppingCallback)

In [4]:
# load dataset
dataset = load_dataset("qanastek/ELRC-Medical-V2", "en-fr")
# Create a proper train/test split to avoid contamination
ds = dataset["train"].train_test_split(test_size=0.2, seed=42) if "train" in dataset else dataset.train_test_split(test_size=0.2, seed=42)

In [5]:
# load pretrained model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # Create separate base model for comparison

In [6]:
def tokenize_fn(examples):
    en_texts = [ex["en"] for ex in examples["translation"]]
    fr_texts = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(en_texts, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(fr_texts, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_ds = ds.map(tokenize_fn, batched=True)

In [8]:
# data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [10]:
# training arguments with early stopping
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Add this line to match evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    load_best_model_at_end=True,  # Enable loading the best model at the end of training
    metric_for_best_model="eval_bleu",  # Use BLEU for early stopping
    greater_is_better=True,  # Higher BLEU is better
)

In [15]:
# load multiple metrics
bleu_metric = evaluate.load("sacrebleu")
meteor_metric = evaluate.load("meteor")
bertscore_metric = evaluate.load("bertscore")

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Post-process text
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[lbl.strip()] for lbl in decoded_labels]
    
    # Calculate all metrics
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor_metric.compute(predictions=decoded_preds, references=[l[0] for l in decoded_labels])
    bertscore_result = bertscore_metric.compute(predictions=decoded_preds, references=[l[0] for l in decoded_labels], lang="fr")
    
    # Return all metrics
    return {
        "bleu": bleu_result["score"],
        "meteor": meteor_result["meteor"],
        "bertscore_f1": np.mean(bertscore_result["f1"])
    }

In [17]:
# Evaluate base model before fine-tuning
base_trainer = Seq2SeqTrainer(
    model=base_model,
    args=training_args,
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_3223/111762798.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  base_trainer = Seq2SeqTrainer(


In [18]:
print("Evaluating base model before fine-tuning...")
base_eval_results = base_trainer.evaluate()
print("Base model metrics:", base_eval_results)

Evaluating base model before fine-tuning...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Base model metrics: {'eval_loss': 0.7234706282615662, 'eval_model_preparation_time': 0.0043, 'eval_bleu': 32.999934595764664, 'eval_meteor': 0.5270604436231981, 'eval_bertscore_f1': 0.8239030855916752, 'eval_runtime': 473.5094, 'eval_samples_per_second': 5.554, 'eval_steps_per_second': 0.348}


In [19]:
# setup trainer for fine-tuning with early stopping
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Add early stopping
)

/tmp/ipykernel_3223/1681039089.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Meteor,Bertscore F1
1,0.635400,0.549707,36.972185,0.638258,0.864561
2,0.531700,0.535434,40.855252,0.641706,0.866834
3,0.478700,0.535532,45.469537,0.668619,0.878935


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
# evaluate the fine-tuned model on the test set
print("Evaluating fine-tuned model...")
eval_results = trainer.evaluate()
print("Fine-tuned model metrics:", eval_results)

In [ ]:
# get a few original examples from the test set (for english and reference french)
sample_count = 5
sample_original = ds["test"].select(range(sample_count))
sample_tokenized = tokenized_ds["test"].select(range(sample_count))

In [ ]:
# get predictions from the fine-tuned model
ft_predictions = trainer.predict(sample_tokenized)
ft_decoded_preds = tokenizer.batch_decode(ft_predictions.predictions, skip_special_tokens=True)

In [ ]:
# get predictions from the base model
base_predictions = base_trainer.predict(sample_tokenized)
base_decoded_preds = tokenizer.batch_decode(base_predictions.predictions, skip_special_tokens=True)

In [ ]:
# print comparisons: original english, fine-tuned pred, base model pred, and reference french
for i, example in enumerate(sample_original):
    english_text = example["translation"]["en"]
    ref_text = example["translation"]["fr"]
    print(f"example {i}:")
    print("english      :", english_text)
    print("sft prediction:", ft_decoded_preds[i])
    print("base prediction:", base_decoded_preds[i])
    print("reference    :", ref_text)
    print("-" * 50)